In [1]:
import warnings
warnings.filterwarnings('ignore')

# Lab 4 - Data (ETL)

In [2]:
%matplotlib inline

## General Instructions

In this course, Labs are the chance to applying concepts and methods discussed in the module.
They are a low stakes (pass/fail) opportunity for you to try your hand at *doing*.
Please make sure you follow the general Lab instructions, described in the Syllabus.
The summary is:

* Discussions should start as students work through the material, first Wednesday at the start of the new Module week. 
* Labs are due by Sunday. 
* Lab solutions are released Monday.  
* Post Self Evaluation and Lab to Lab Group on Blackboard and Lab to Module on Blackboard on Monday.

The last part is important because the Problem Sets will require you to perform the same or similar tasks without guidance.
Problem Sets are your opportunity to demonstrate that you understand how to apply the concepts and methods discussed in the relevant Modules and Labs.

## Specific Instructions

1.  For Blackboard submissions, if there are no accompanying files, you should submit *only* your notebook and it should be named using *only* your JHED id: fsmith79.ipynb for example if your JHED id were "fsmith79". If the assignment requires additional files, you should name the *folder/directory* your JHED id and put all items in that folder/directory, ZIP it up (only ZIP...no other compression), and submit it to Blackboard.

    * do **not** use absolute paths in your notebooks. All resources should located in the same directory as the rest of your assignments.
    * the directory **must** be named your JHED id and **only** your JHED id.
    * do **not** return files provided by us (data files, .py files)

2. Data Science is as much about what you write (communicating) as the code you execute (researching). In many places, you will be required to execute code and discuss both the purpose and the result. Additionally, Data Science is about reproducibility and transparency. This includes good communication with your team and possibly with yourself. Therefore, you must show **all** work.

3. Avail yourself of the Markdown/Codecell nature of the notebook. If you don't know about Markdown, look it up. Your notebooks should not look like ransom notes. Don't make everything bold. Clearly indicate what question you are answering.

4. Submit a cleanly executed notebook. The first code cell should say `In [1]` and each successive code cell should increase by 1 throughout the notebook.

**Note** This assignment will have multiple files. Follow those instructions.

## Lab

**Reid's** is a small breakfast stand that sells drinks (coffee, tea, sodas) and food (egg & sausage, oatmeal) in a commercial downtown area, Monday through Friday, from 8a until 11am.
Although their menu is small, they do try to cater to a wide variety of diets and thus provide both vegan and keto options for most of their meals.
They started using Ordr as their Point of Sale system about two months ago and are on the Basic Plan.

Under the Basic Plan, they are able to use the Ordr API to access orders.
This order information comes in the form of a denormalized JSON document.
In order to make any sense of things, you need to normalize it in the Datawarehouse.

1. You are not actually going to access an external API. Use the provided JSON file as the data that the API would return.
2. You are not actually going to use a datawarehouse. Instead you will use a SQLite database.

**Note** We sometimes get strange questions about the use of SQLite like, "do you really use SQLite in production?". We use SQLite jor this Lab for the following reasons:

1. SQLite is a real RDBMS.
2. SQLite uses real SQL. SQL may be the most important skill you can have as a Data Scientist doing Data Science.
3. Most importantly, the database itself is a standalone file that you can submit to us.

That being said, under some and somewhat weird circumstances, I have used SQLite on real projects before. However, the learning objective is not SQLite, SQLite is  tool.

**Note**
We assume you know the basics of RDMBS in this course.
That you understand what "normalized" and "denormalized" data means and that you know about primary and foreign keys.
This [article](https://www3.ntu.edu.sg/home/ehchua/programming/sql/Relational_Database_Design.html) does talk about the major points.
Additionally, we assume you know SQL and DDL.
If you do not, this Lab will be more challenging than usual and you should start early.

**Important - You must not use Pandas for any part of this assignment.**
Why not?
Because you should know how to do these things without relying on Pandas.

## Part 1

### Learning Objectives

* investigate the structure of data acquired from a 3rd party.
* convert denormalized data into normalized data, according to common data warehouse practices.
* design an RDBMS database
* write data to a datastore (RDMBS).

This assignment is not about tools *per se* but about broader skills and concepts.

You will be creating the following files:

1. **reids.sql** - this file will create the database structure using DDL. Make sure you review the data and sketch out your design.
2. **reids.db** - this is the actual database.

```
> sqlite3 reids.db < reids.sql
```

will create the database and all the tables.
The database will be empty at this point.

3. **reids.py** - this program will parse the JSON file and fill the database.

```
> python reids.py
```

Unfortunately, the documentations for the Ordr API is sparse, here is an example of one order:

```
{'items': [
    {'name': 'coffee', 'price': 2.75},
    {'name': 'flavor shot', 'price': 1.0}
    ],
'charges': {
    'date': '01/04/21 10:22',
    'subtotal': 3.75,
    'taxes': 0.26,
    'total': 4.01},
'payment': {
    'card_type': 'visa',
    'last_4_card_number': '0465',
    'zip': '21217',
    'cardholder': 'Christina Sampson',
    'method': 'credit_card'}}
```

**The date format is Month/Day/Year**

Make sure you look through the data to see what values are possible for each of the fields.
The standards of normalization/denormalization for datawarehouses are slightly different or can be different than regular production RDMBS systems.
For example, we might be tempted to create a `menu_items` table:

```
id    name                 price
1     coffee               2.75
2     flavor shot          1.00
3     egg salad sandwich   4.50
```

An issue arises if we change the name to "Kona Coffee" because it will change it *for past purchases*.
That is, customers in the past bought based on the name "Coffee" and not "Kona Coffee".
This might be important.
Even worse, if we change the price to \\$3.00, it changes it for all past purchases and that is clearly wrong.

In a production RDMBS we often want the data to change everywhere it is used.
If "Steve" changes his name to "Sam", we want that to be reflected in any query and report.
For datawarehousing, though, we want to preserve historical fidelity of the data.
This means we have a tendency to normalize *less* than we would otherwise do.
It's worth noting that there is trend to preserve the historical fidelity of production database as well by things like soft deletes.

This means the main issue for the Ordr data is storing the three main entities and creating primary/secondary keys.
You will need to create these.

All of this "parsing and massaging" work will be done in the `reids.py` file.
It will contain the code to parse the JSON file and fill the database, performing whatever normalization and standardization is required as well as creating whatever primary and foreign key relationships seem reasonable.

You must create the following tables in the database:

1. `items`
2. `charges`
3. `payments`

but you can add additional tables as necessary (it is not uncommmon to include tables in datawarehouses that support analytics such as information about business dates).

**Note** Feel free to develop reids.py as a Notebook and then generate the .py from the .ipynb file...just make sure you only include the .py file and that it will run from the command line as specified above and you have commented out any debug/chatter.

**Important**
There are some "gotchas".
1. When inserting data into the database, don't forget to the commit.
2. If you must reconstruct your database, make sure you "free" all references to it. If you use a script to change it but it's open your notebook, the open version in the notebook won't necessarily see those changes.  You'll need to get a new connection.

When you are done with this part, you should be able to proceed to Part 2.

## Part 2

**Everything having to do with parsing the JSON file from Ordr and setting up the "datawarehouse" should be done in the three files described above and not in this Notebook.**

### Learning Objectives

You almost never start out with a Notebook and start pulling data. The idea that you launch Jupyter Notebook and load a readidly available CSV is an incredibly artificial artifact of school (if you had to pull data The Real Way(tm) for every assignment, we'd never get anything done).

Instead, you are more likely to start out with a database and you run queries directly against the database, finding out where and what everything is, answering some initial questions.

* Run queries against an RDBMS to answer basic business questions.

Some data science projects are literally just this: someone asks a question, you investigate the data, you run a query using something like [MySQL Workbench](https://www.mysql.com/products/workbench/), [Toad](https://www.toadworld.com/products/toad-for-sql-server) (Windows Only) or [Postico](https://eggerapps.at/postico/) (MacOS Only). There are also generic SQL clients. For example, [VSCode](https://code.visualstudio.com/) has SQL extensions.

You will mimic that experience here by using only the [sqlite3](https://docs.python.org/3/library/sqlite3.html) Python library (included in the base installiation, link is to documentation).
As with Part 1, you may *not* use Pandas for this part.
Additionally, you *must* not print out native Python data structures.
[Tabulate](https://pypi.org/project/tabulate/) has been provided in the environment.yml for your use.


For Part 2, everything should be done here, in this notebook.

**Note**
The general format is discuss/code/discuss.
For the questions below, you should be able to:

1. explain what the query does (discuss)
2. execute and display the query result (code)
3. interpret the result (discuss)

All three are required for full credit on something like Problem Set so you should practice the triad here. It is permissible to use a query to get raw data (and show it in a table) and then perform a calculation with that raw data (just add a code cell). However, you should do as much as possible in SQL.

Using the database `reids.db` and SQL please answer the following questions:

In [3]:
from tabulate import tabulate
import sqlite3

In [4]:
con = sqlite3.connect('reids.db')
cursor = con.cursor()

### Question 1.

What were Reid's order count and gross revenue by day for the two month period?




First thing I want to do is confirm that indeed the orders range for a two month period

In [5]:
print('First purchased was on: ')
print(*cursor.execute('select date from charges order by date limit 1').fetchone())
print('\nLast purchased was on: ')
print(*cursor.execute('select date from charges order by date desc limit 1').fetchone())

First purchased was on: 
2021-04-01

Last purchased was on: 
2021-05-28


Indeed, the purchases do span a two month period. Next, we can simply use a SQL query to group the dates together to tally up the number of orders made that day as well as its gross revenue.

In [6]:
data = cursor.execute("select date, count(date), sum(total) from charges group by date;")
print(tabulate(data.fetchall(), headers=["Day",'Order Count', 'Gross Revenue'], tablefmt='psql'))

+------------+---------------+-----------------+
| Day        |   Order Count |   Gross Revenue |
|------------+---------------+-----------------|
| 2021-04-01 |            34 |          188.87 |
| 2021-04-02 |            51 |          265.11 |
| 2021-04-05 |            57 |          339.77 |
| 2021-04-06 |            48 |          276.09 |
| 2021-04-07 |            32 |          188.63 |
| 2021-04-08 |            58 |          345.92 |
| 2021-04-09 |            62 |          341.58 |
| 2021-04-12 |            55 |          294.54 |
| 2021-04-13 |            32 |          167.5  |
| 2021-04-14 |            54 |          288.41 |
| 2021-04-15 |            45 |          262.69 |
| 2021-04-16 |            75 |          402.63 |
| 2021-04-19 |            49 |          276.1  |
| 2021-04-20 |            43 |          265.65 |
| 2021-04-21 |            39 |          207.62 |
| 2021-04-22 |            55 |          316.76 |
| 2021-04-23 |            75 |          408.49 |
| 2021-04-26 |      

I verified this result by hand. I queried `select total from charges where date = '04/01/21'` and sum up the total sales and count. Results were consistent with the table above. From this, one further step to do is see what day of the week the best sales fall under e.g 05/25/21 is a weekend or a Wed? One interpretation we can get is to identity when Reids should really be open to maximize revenue.

### Question 2.

What is Reid's average order count and gross revenue by day of the week?




From my above discussion, this got me interested to see if Reids is open on weekends at all. Let's use strftime to find out. The numbers correspond to the day of the week e.g 0 is Sunday, 1 is Monday, etc.

In [7]:
data = cursor.execute("select distinct date, strftime('%w', date) from charges")
print(tabulate(data.fetchall(), headers=["Day",'Day of the Week'], tablefmt='psql'))

+------------+-------------------+
| Day        |   Day of the Week |
|------------+-------------------|
| 2021-04-01 |                 4 |
| 2021-04-02 |                 5 |
| 2021-04-05 |                 1 |
| 2021-04-06 |                 2 |
| 2021-04-07 |                 3 |
| 2021-04-08 |                 4 |
| 2021-04-09 |                 5 |
| 2021-04-12 |                 1 |
| 2021-04-13 |                 2 |
| 2021-04-14 |                 3 |
| 2021-04-15 |                 4 |
| 2021-04-16 |                 5 |
| 2021-04-19 |                 1 |
| 2021-04-20 |                 2 |
| 2021-04-21 |                 3 |
| 2021-04-22 |                 4 |
| 2021-04-23 |                 5 |
| 2021-04-26 |                 1 |
| 2021-04-27 |                 2 |
| 2021-04-28 |                 3 |
| 2021-04-29 |                 4 |
| 2021-04-30 |                 5 |
| 2021-05-03 |                 1 |
| 2021-05-04 |                 2 |
| 2021-05-05 |                 3 |
| 2021-05-06 |      

As you can see, the numbers range from 1 to 5, or Monday to Friday. Reids does not open on Saturday or Sunday! Let us go ahead and calculate the average order count and gross revenue by day of the week.

In [8]:
data = cursor.execute("""select x, round(avg(z)), avg(y) from (select distinct date, strftime('%w', date) as x, sum(total) 
    as y, count(date) as z from charges group by date order by x) group by x""")
print(tabulate(data.fetchall(), headers=["Day",'Day of the Week', "Revenue"], tablefmt='psql'))

+-------+-------------------+-----------+
|   Day |   Day of the Week |   Revenue |
|-------+-------------------+-----------|
|     1 |                48 |   276.631 |
|     2 |                44 |   247.23  |
|     3 |                41 |   232.758 |
|     4 |                51 |   292.967 |
|     5 |                60 |   325.244 |
+-------+-------------------+-----------+


Friday is the most profitable day!

### Question 3.

How many cups of coffee does Reid's sell per day, on average?




My idea here is to find the total number of days that Reid's opens because it definitely doesn't open everyday. For example, it does not open on 04/03/21. Once we find this number, we can use it to divide the total number of coffee sold. This will give us an average of how many cups of coffee were sold in average of the number of days the store was open.

In [9]:
total_selling_day = cursor.execute('select count(distinct date) from charges').fetchone()[0]
print(f"Reids opens for {total_selling_day} total number of days")

total_coffee_sold = cursor.execute('select count(*) from items where name = "coffee"').fetchone()[0]
print(f"Reids sold {total_coffee_sold} cups of coffee in total during the span of two months")

print(f'In average, Reid\'s sold about {total_coffee_sold/total_selling_day:0.0f} cups of coffee per day')

Reids opens for 42 total number of days
Reids sold 1516 cups of coffee in total during the span of two months
In average, Reid's sold about 36 cups of coffee per day


Preliminary algebra tells us that 36 cups of coffee are sold per day in average. Let us analyze this further with some queries.

In [10]:
data = cursor.execute("select distinct i.name, c.date, count(c.date) from charges c join items i where \
                      c.ordersID = i.ordersID and i.name = 'coffee' group by c.date")
print(tabulate(data.fetchall(), headers=["Item Name",'Purchased Date', 'Number of Coffee Bought'], tablefmt='psql'))

+-------------+------------------+---------------------------+
| Item Name   | Purchased Date   |   Number of Coffee Bought |
|-------------+------------------+---------------------------|
| coffee      | 2021-04-01       |                        30 |
| coffee      | 2021-04-02       |                        31 |
| coffee      | 2021-04-05       |                        36 |
| coffee      | 2021-04-06       |                        34 |
| coffee      | 2021-04-07       |                        25 |
| coffee      | 2021-04-08       |                        43 |
| coffee      | 2021-04-09       |                        50 |
| coffee      | 2021-04-12       |                        37 |
| coffee      | 2021-04-13       |                        21 |
| coffee      | 2021-04-14       |                        39 |
| coffee      | 2021-04-15       |                        35 |
| coffee      | 2021-04-16       |                        56 |
| coffee      | 2021-04-19       |                     

The distinctive count of this table is 42. This tells me that coffee was purchases in every day that Reid's was open since we learned from above that the store opens 42 days out of the two months.

The third column shows how many coffees were bought that day. Let us find the average.

In [11]:
num_coffee_bought_per_day = [30,31,36,34,25,43,50,37,21,39,35,56,35,35,26,46,57,33,32,
                             32,36,37,36,36,18,49,44,37,25,27,47,38,22,31,38,50,41,35,41,29,30,36,]

sum(num_coffee_bought_per_day) / len(num_coffee_bought_per_day)

36.095238095238095

This agrees with our math earlier and concludes the exercise that 36 cups of coffee are sold per day in average.

### Question 4.

What proportion of orders contain "up charges" like flavor shots, vegan or keto substitutions?




The strategy here is to tally up every order of all items and then find the ratio relative to the said substitutions.

In [12]:
data = cursor.execute("select distinct name, price from items")
print(tabulate(data.fetchall(), headers=["Item Name",'Price'], tablefmt='psql'))

+---------------+---------+
| Item Name     |   Price |
|---------------+---------|
| coffee        |    2.75 |
| espresso      |    1.25 |
| flavor shot   |    1    |
| egg & sausage |    4.75 |
| oatmeal       |    3    |
| pastry        |    2.5  |
| tea           |    1.25 |
| soda          |    2.5  |
| keto          |    1.75 |
| vegan         |    1.25 |
| cookie        |    2.5  |
| water         |    1    |
+---------------+---------+


There are twelve unique items on the menu and three of them are substitutions (flavor shot, vegan, and keto).

In [13]:
flavor_shot = cursor.execute('select count(*) from items where name = "flavor shot"').fetchone()[0]
print(f'Total number of purchases on flavor shot items: {flavor_shot}')
vegan = cursor.execute('select count(*) from items where name = "vegan"').fetchone()[0]
print(f'Total number of purchases on vegan items: {vegan}')
keto = cursor.execute('select count(*) from items where name = "keto"').fetchone()[0]
print(f'Total number of purchases on keto items: {keto}')

count = cursor.execute('select count(*) from items').fetchone()[0]
print(f"Out of {count} item purchases, {flavor_shot+vegan+keto} are 'up charges' orders" )

Total number of purchases on flavor shot items: 114
Total number of purchases on vegan items: 236
Total number of purchases on keto items: 103
Out of 4145 item purchases, 453 are 'up charges' orders


453 out of 4145 yields a proportion of 10.9%. Guess one out of ten people is health conscious!

### Question 5.

Reid's considers someone to be a "regular" if they come at least 3 out of 5 days per week. How many regulars do you estimate there are and what are their names? How many days per week do they each come on average? What are the limits of this calculation based on the available data?




First off, the limitation is the information on the customer's name. We are only provided their name if they use their credit card, we do not know the name of those that paid in cash.

Here is my thought process:
1. Join the charges and payment tables together to examine the frequency of purchases by the same customer
  * Only query when payment type is credit and skip when type is cash
2. Group by customer name and tally up the count for the week(s)
3. Derive a conclusion on the regular customers
4. Find average of customers' visit by summing up visits per week divided by total number of weeks spanning those 42 days

In [14]:
data = cursor.execute("""select x.date, y.name from (select c.date as date, p.customersID from charges c join payment p 
    where c.ordersID = p.ordersID and p.method = 'credit_card') as x join customers y 
    where x.customersID = y.customersID order by y.name""")
print(tabulate(data.fetchall(), headers=['Purchased Date', 'Cardholder'], tablefmt='psql'))

+------------------+------------------------+
| Purchased Date   | Cardholder             |
|------------------+------------------------|
| 2021-04-07       | Aaron Bridges          |
| 2021-04-20       | Aaron Compton          |
| 2021-05-13       | Aaron Crane            |
| 2021-04-26       | Aaron Ellis            |
| 2021-04-23       | Abigail Carter         |
| 2021-04-09       | Abigail Hall           |
| 2021-05-28       | Adam Brooks            |
| 2021-04-08       | Adam Thompson          |
| 2021-04-06       | Adrian Herman          |
| 2021-04-20       | Adrian James           |
| 2021-05-13       | Adriana Young          |
| 2021-04-05       | Alexander Atkinson     |
| 2021-04-26       | Alexander Nelson       |
| 2021-04-16       | Alexis Campbell        |
| 2021-04-26       | Alexis Thompson        |
| 2021-04-27       | Allison White          |
| 2021-04-08       | Allison Wright         |
| 2021-04-08       | Alyssa Carr            |
| 2021-04-16       | Alyssa Marsh 

The table above is ordered by the cardholders. It shows us when each customer (that pays with credit card) purchased e.g Andrew Luna bought something on 04/01/21 and again on 04/05 and 04/07 etc.

The table below is grouped by the cardholders. It shows us how many times in total the customers purchased with their credit card in the 42 days that Reid's opened. For example, Andrew Luna purchased 29 times.

I think the real challenge here is to also determine if the customer swiped their card on the same given day. This is problematic because the question is asking for a frequency of 3 out of 5 business days.

In [15]:
data = cursor.execute("""select y.name, count(y.name) from (select c.date as date, p.customersID from 
    charges c join payment p 
    where c.ordersID = p.ordersID and p.method = 'credit_card') as x join customers y 
    where x.customersID = y.customersID group by y.name""")

print(tabulate(data.fetchall(), headers=['Cardholder', 'Frequency'], tablefmt='psql'))

+------------------------+-------------+
| Cardholder             |   Frequency |
|------------------------+-------------|
| Aaron Bridges          |           1 |
| Aaron Compton          |           1 |
| Aaron Crane            |           1 |
| Aaron Ellis            |           1 |
| Abigail Carter         |           1 |
| Abigail Hall           |           1 |
| Adam Brooks            |           1 |
| Adam Thompson          |           1 |
| Adrian Herman          |           1 |
| Adrian James           |           1 |
| Adriana Young          |           1 |
| Alexander Atkinson     |           1 |
| Alexander Nelson       |           1 |
| Alexis Campbell        |           1 |
| Alexis Thompson        |           1 |
| Allison White          |           1 |
| Allison Wright         |           1 |
| Alyssa Carr            |           1 |
| Alyssa Marsh MD        |           1 |
| Amanda Alexander       |           1 |
| Amanda Anderson        |           1 |
| Amanda Ayala  

Let's clean this up. The above table shows extraneous records with first time customers. Let's filter out the customers who have visited less than 3 time.

A random side note, it is interesting that customers are either die hard or just passing by. We see that the frequency is either 1 or a number at least 20!

In [16]:
data = cursor.execute("""select y.name, count(y.name) from (select c.date as date, p.customersID from 
    charges c join payment p 
    where c.ordersID = p.ordersID and p.method = 'credit_card') as x join customers y 
    where x.customersID = y.customersID group by y.name having count(y.name) > 3""")
print(tabulate(data.fetchall(), headers=['Cardholder', 'Frequency'], tablefmt='psql'))

+-------------------+-------------+
| Cardholder        |   Frequency |
|-------------------+-------------|
| Andrea Baker DVM  |          29 |
| Andrea Francis    |          27 |
| Andrew Luna       |          29 |
| Anthony Martin    |          34 |
| Barbara Greene    |          30 |
| Brent Morgan      |          30 |
| Calvin Jones      |          31 |
| Cassandra Francis |          33 |
| Cheryl Zamora     |          30 |
| Craig Armstrong   |          29 |
| Craig Chase       |          30 |
| Daniel Long       |          27 |
| David Jordan      |          29 |
| Emily Dennis      |          32 |
| Eric Bruce        |          33 |
| Gina Green        |          34 |
| Gregory Jones     |          33 |
| Jamie Smith       |          27 |
| Jason Barker      |          28 |
| Karen Sutton      |          27 |
| Lindsey Hammond   |          30 |
| Lisa Aguilar      |          34 |
| Margaret Brown    |          31 |
| Mary Jones        |          33 |
| Matthew Lewis     |       

Next thing to do here is to a join between these loyal customers' ID and the charges table that has the information on the payment dates. From there we'll want to extract the day of the week using strftime(%W, date) to return the week of the year. This information can be used to check if the customer frequently visits Reids at least 3 out of the 5 weekdays.